Para calcular el pageRank de cualquier matriz se pide emplear una funcion llamada pageRank que reciba como argumentos la matriz G y un damping factor seteado por defecto en 0,85. Para esto primero importamos las librerías a utilizar


---

*Para efectos de google.colab se deben 'correr' todas las celdas de código en orden, si no arrojará error ya que no lee celdas anteriores*

In [ ]:
import numpy as np
import scipy.linalg as la

Luego se crea la función pageRank. El funcionamiento de la función se explica comentado en el código.

In [ ]:
#primero se define la función con el nombre y argumentos pedidos
def pageRank(G, d=0.85): 
  #Luego se necesita calcular la suma de todos los pesos por cada columna
  #para eso se va a crear una lista llamada suma_columnas donde en cada 
  #columna de esta lista se almacena la suma de los pesos por columnas de todas
  #las filas de la columna G ingresada a la función
  suma_columnas = []

  #En los siguientes ciclos for se va a recorrer toda la matriz (filas -> i, columnas -> j)
  for i in range(len(G)):
    suma = 0 #se crea un acumulador donde se iran guardando la suma de los pesos
    for j in range(len(G)):
      suma = suma + G[j][i]
    suma_columnas.append(suma) #la suma final se agrega en la lista

  #luego vamos a recorrer la matriz nuevamente cambiando los valores de la matriz
  for fila in range(len(G)):
    for columna in range(len(G)):
      #si el valor en la matriz original era 0,
      #ahora se cambia a (1-d)/n donde n es la cantidad de nodos
      if G[fila][columna] == 0: 
        G[fila][columna] = (1-d)/len(G)
      #si el valor de la matriz original es distinta de cero se cambia ese valor por
      #(1-d)/n + Win*d/Sum_W donde Win es el peso del link que sale del nodo j y entra al nodo i(valor de la matriz en es punto)
      #y Sum_W es la suma de los pesos de todos los links que salen del nodo j(suma de todos los pesos de esa columna)
      elif G[fila][columna] != 0:
        G[fila][columna] = (1-d)/len(G) + G[fila][columna]*d/suma_columnas[columna]

  #Asi podemos reescribir la matriz G automaticamente sumando las contribuciones del damping factor
  #por lo que podemos calcular el PageRank tal como lo vimos en clases
  valG, UG = la.eig(G) 
  w = UG[:, 0]
  pr = np.absolute(UG[:, 0]/la.norm(UG[:, 0], 1))
  return pr

Para probar que está bien, definimos la matriz G dada en el enunciado de la tarea y la probamos con un damping factor de 0,85 y 0,1 obteniendo aproximadamente los mismos resultados mostrados en la tarea. Se puede ver que los resultados en comparación a la tarea son similares, solo tienen una diferencia de +- 0.001 (1%) lo que consideramos despreciable para esta aplicación.

In [ ]:
G = np.array([[0, 1, 0, 0, 0, 0, 0, 0],
              [1, 0, 0, 1, 0, 0, 0, 0],
              [0, 1, 0, 0, 1, 1, 1, 1],
              [0, 0, 0, 0, 1, 0, 1, 1],
              [0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 1, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1, 0, 0, 0]
              ], dtype=float)

print(pageRank(G))
print(pageRank(G, .1))

[0.05141623 0.07686172 0.37997977 0.0508515  0.03315793 0.34877886
 0.01875    0.04020399]
[0.11982156 0.12815428 0.14327974 0.12686303 0.11796945 0.12906063
 0.114375   0.1204763 ]


A continuación probamos el caso donde se cambian las ponderaciones para cambiar la importancia de los nodos, obteniendo los resultados esperados

In [ ]:
H = np.array([[0, 1, 0, 0, 0, 0, 0, 0],
              [1, 0, 0, 1, 0, 0, 0, 0],
              [0, 0.001, 0, 0, 0.001, 0.001, 0.001, 0.001],
              [0, 0, 0, 0, 1, 0, 1, 1],
              [0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 1, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1, 0, 0, 0]
              ], dtype=float)

print(pageRank(H))

[0.21012051 0.22536691 0.16655023 0.09887465 0.04676448 0.17356322
 0.01875    0.06001001]


Finalmente para la pregunta 2.4 se debe utilizar la función pageRank descrita anteriormente con 2 datasets dados para calcular el nodo con mayor pageRank en cada caso.

Para esto se debe utilizar la función dada llamada fromFiletoMatrix donde se le entrega un archivo .txt devolviendo una matriz que se puede ingresar en la función pageRank.



---



*Para efectos específicos del uso de google.colab se deben subir los archivos del dataset cada vez que se quiera ejecutar el código, ya que se está ejecutando en la nube y no google.colab guarda archivos. Además se debe copiar la ruta del archivo tal como esta en el código al aplicarlo en la función fromFiletoMatrix*



In [ ]:
#La funcion fromFiletoMatrix recibe como argumentos el archivo que contiene el grafo dirigido 
#(como lista de arcos) y la cantidad de nodos del grafo y retorna
#la representación como matriz del grafo (traspuesta de la matriz de adyacencia)
def fromFiletoMatrix(file, num_nodes):
    G = np.zeros((num_nodes, num_nodes))
    f = open(file, 'r')
    for line in f:
        arc = line.split()
        G[int(arc[1])-1, int(arc[0])-1] = 1.0
    f.close()
    return G

#Primero calculamos el pageRank de cada matriz
PR1 = pageRank(fromFiletoMatrix('/content/email-Eu-core-temporal-norm.txt', 986))
PR2 = pageRank(fromFiletoMatrix('/content/wiki-Vote-norm.txt', 7115))

#Además podemos encontrar el valor del máximo pageRank de cada dataset
maxPR1 = max(PR1)
maxPR2 = max(PR2)

#Finalmente para encontrar el nodo en el que está ubicado el mayor pageRank podemos usar la funcion np.where de numpy
nodo1 = np.where(max(PR1) == PR1)
nodo2 = np.where(max(PR2) == PR2)

#imprimimos en pantalla el valor del nodo con pageRank mayor (página más recomendada)
#Al imprimir en pantalla se debe sumar uno ya que la lista parte desde 0
print(nodo1[0]+1)
print(nodo2[0]+1)

FileNotFoundError: ignored